In [1]:
import numpy as np
import json, glob
import pandas as pd
#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy
import spacy
from nltk.corpus import stopwords


# vis
import pyLDAvis
import pyLDAvis.gensim


# full = pd.read_csv("/home/penguin/Projects/SKIPSIE/_RESEARCH/processed.csv")

In [2]:
stopwords = stopwords.words('english')

In [3]:
print(stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

# get data

In [4]:
data = pd.read_csv("/home/penguin/Projects/SKIPSIE/DATA/processed.csv")
grouped = data.groupby("label")
stress_posts = grouped.get_group(1)['text']
non_stress_posts = grouped.get_group(0)['text']


In [5]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"]) #small spacy model, disabling parser and named entity recognition, more compu expensive.
    texts_out = []
    for text in texts:
        doc=nlp(text)
        new_text=[]
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)

lemmatized_texts_stress = lemmatization(stress_posts)
lemmatized_texts_nonstress = lemmatization(non_stress_posts)
print(lemmatized_texts_stress[0][0:90])

say feel way suggete go rest trigger ahead youire decide look feel doom hope maybe get suc


In [6]:
#  function from the Gensim library.
# The function simple_preprocess is used to preprocess text by converting it to lowercase,
#  tokenizing it, and removing punctuation marks. as well as stopwords
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return final

stress_words = gen_words(stress_posts)
non_stress_words = gen_words(non_stress_posts)


In [66]:
# BIGRAMS AND TRIGRAMS
# threshold = amount of phrases, min_count = the necessary amount of co-occurences of word pairing.
bigrams_phrases_stress = gensim.models.Phrases(stress_words,min_count=5, threshold=100)
trigram_phrases_stress = gensim.models.Phrases(bigrams_phrases_stress[stress_words],threshold=100)
bigrams_phrases_non_stress = gensim.models.Phrases(non_stress_words,min_count=5, threshold=50)
trigram_phrases_non_stress = gensim.models.Phrases(bigrams_phrases_non_stress[non_stress_words],threshold=50)

bigram_stress = gensim.models.phrases.Phraser(bigrams_phrases_stress)
trigram_stress = gensim.models.phrases.Phraser(trigram_phrases_stress)

bigram_non_stress = gensim.models.phrases.Phraser(bigrams_phrases_non_stress)
trigram_non_stress = gensim.models.phrases.Phraser(trigram_phrases_non_stress)

def make_bigrams(bigram, texts):
    return(bigram[doc] for doc in texts)

def make_trigram(trigram,bigram, texts):
    return(trigram[bigram[doc]] for doc in texts)


data_bigrams_stress = make_bigrams(bigram_stress,stress_words)
data_bigrams_trigrams_stress = make_trigram(trigram_stress,bigram_stress, data_bigrams_stress)

data_bigrams_non_stress = make_bigrams(bigram_non_stress,non_stress_words)
data_bigrams_trigrams_non_stress = make_trigram(trigram_non_stress,bigram_non_stress, data_bigrams_non_stress)

data_bigrams_trigrams_stress = list(data_bigrams_trigrams_stress)
data_bigrams_trigrams_non_stress = list(data_bigrams_trigrams_non_stress)
print(data_bigrams_trigrams_stress[0][0:20])
print(data_bigrams_trigrams_non_stress[0][0:20])
# for i, doc in enumerate(data_bigrams_trigrams_stress):
#     if i >= 20:
#         break
#     print(doc)

['said', 'felt', 'way', 'suggeted', 'go', 'rest', 'trigger', 'ahead', 'youire', 'hypocondriac', 'like', 'decide', 'look', 'feeling', 'doom', 'hope', 'maybe', 'getting', 'sucked', 'rabbit']
['hey', 'rassistance', 'sure', 'right', 'place', 'post', 'go', 'im', 'currently', 'student', 'intern', 'sandia', 'national', 'lab', 'working', 'survey', 'help', 'improve', 'marketing', 'outreach']


In [70]:
# TF - IDF REMOVAL
from gensim.models import TfidfModel

id2word = corpora.Dictionary(data_bigrams_trigrams_stress)
texts = data_bigrams_trigrams_stress
corpus = [id2word.doc2bow(text) for text in texts]
print(corpus[0][0:20])

tfidf = TfidfModel(corpus,id2word=id2word)
low_value=0.03
words=[]
words_missing_in_tf_idf = []

for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = [] #reinitialize to be safe. You can skip this.
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words + words_missing_in_tf_idf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]          
    corpus[i] = new_bow
# https://stackoverflow.com/questions/24688116/how-to-filter-out-words-with-low-tf-idf-in-a-corpus-with-gensim/35951190


[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 3), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1)]


In [25]:
# above newer version of this.
# id2word_stress = corpora.Dictionary(stress_words)
# id2word_non_stress = corpora.Dictionary(non_stress_words)

# def id2word_corpus(corpus,id2word, posts):

#     corpus = []
#     for text in posts:
#         new = id2word.doc2bow(text)
#         corpus.append(new)
#     return corpus,id2word


# corpus_stress , id2word_stress = id2word_corpus([],id2word_stress,stress_words)
# corpus_non_stress,id2word_non_stress = id2word_corpus([], id2word_non_stress, non_stress_words)

In [77]:
lda_model_stress = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=10,
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha='auto')
# lda_model_non_stress = gensim.models.ldamodel.LdaModel(corpus=corpus_non_stress,
#                                             id2word=id2word_non_stress,
#                                             num_topics=10,
#                                             random_state=100,
#                                             update_every=1,
#                                             chunksize=100,
#                                             passes=10,
#                                             alpha='auto')


In [79]:
test_doc = corpus[-1]
vector = lda_model_stress[test_doc]
print(vector)

def Sort(sub_li):
    sub_li.sort(key = lambda x: x[1])
    sub_li.reverse()
    return (sub_li)

new_vector = Sort(vector)
print(new_vector)

[(1, 0.14147572), (3, 0.39710686), (4, 0.024034975), (7, 0.09827868), (8, 0.21473263), (9, 0.10194865)]
[(3, 0.39710686), (8, 0.21473263), (1, 0.14147572), (9, 0.10194865), (7, 0.09827868), (4, 0.024034975)]


In [ ]:
# save and load model (possible later update it.) ... https://www.youtube.com/watch?v=xADAr8pPQMI&list=PL2VXyKi-KpYttggRATQVmgFcQst3z6OlX&index=12

# visualize data

In [75]:
pyLDAvis.enable_notebook()
vis1 = pyLDAvis.gensim.prepare(lda_model_stress,corpus,id2word,mds="mnds",R=30)
vis1

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.262238 -0.145058       1        1  44.951400
1     -0.098951  0.296083       2        1  26.988526
3     -0.059270 -0.085932       3        1  20.659843
0      0.227642 -0.036882       4        1   4.490957
2      0.192817 -0.028211       5        1   2.909275, topic_info=           Term         Freq        Total Category  logprob  loglift
98           wa  1169.000000  1169.000000  Default  30.0000  30.0000
129          im  1295.000000  1295.000000  Default  29.0000  29.0000
24         like   910.000000   910.000000  Default  28.0000  28.0000
120        feel   753.000000   753.000000  Default  27.0000  27.0000
58         dont   718.000000   718.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
775     company     7.438385     8.152386   Topic5  -5.7074   3.4456
2235     effect     8.223774     9.014882   Topic5  -5.6071   3.4454
3670       lyme     7.804147     8.556528   Topic5  -5.6594   3.4452
1467       spot     7.608678     8.345670   Topic5  -5.6848   3.4448
1494  apologize     7.574273     8.322446   Topic5  -5.6893   3.4431

[241 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
957       1  0.011830   abuse
957       2  0.981905   abuse
2351      4  0.942048  across
2831      5  0.898824     add
545       2  0.995422     ago
...     ...       ...     ...
99        1  0.415230   would
99        2  0.451703   would
99        3  0.131864   would
100       2  0.907972    year
100       3  0.090047    year

[285 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 2, 4, 1, 3])

In [39]:
# pyLDAvis.enable_notebook()
# vis2 = pyLDAvis.gensim.prepare(lda_model_non_stress,corpus_non_stress,id2word_non_stress,mds="mnds",R=30)
# vis2

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9      0.317375 -0.076396       1        1  35.283096
5      0.291769 -0.156545       2        1  26.467978
4      0.157703  0.357867       3        1  15.116409
6     -0.088463 -0.028301       4        1   4.749301
8     -0.102643 -0.008801       5        1   4.417248
1     -0.118380 -0.017137       6        1   3.952587
0     -0.111095 -0.023669       7        1   3.234644
2     -0.117847 -0.013600       8        1   2.713215
3     -0.114198 -0.019081       9        1   2.685060
7     -0.114220 -0.014337      10        1   1.380461, topic_info=         Term         Freq        Total Category  logprob  loglift
85         wa  1371.000000  1371.000000  Default  30.0000  30.0000
109       get   471.000000   471.000000  Default  29.0000  29.0000
152      like   540.000000   540.000000  Default  28.0000  28.0000
103      dont   448.000000   448.000000  Default  27.0000  27.0000
20       know   429.000000   429.000000  Default  26.0000  26.0000
...       ...          ...          ...      ...      ...      ...
472   treated     5.134911     5.925391  Topic10  -5.1974   4.1396
6548     bear     4.892464     5.682964  Topic10  -5.2457   4.1330
549   article     4.764494     5.554700  Topic10  -5.2722   4.1293
467      send     7.679746    22.312422  Topic10  -4.7948   3.2162
468     story     6.608040    50.193079  Topic10  -4.9451   2.2552

[400 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
1022      2  0.212240      able
1022      3  0.778214      able
229       2  0.994105     abuse
285       3  0.965716  actually
285       5  0.026825  actually
...     ...       ...       ...
47        3  0.095615     would
1654      6  0.977401        xb
205       1  0.998968      year
850       4  0.979911       yet
3524      4  0.972186   younger

[443 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 6, 5, 7, 9, 2, 1, 3, 4, 8])